In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [146]:
import addict
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

plt.rcParams['text.usetex'] = False # Plotting the global model is much much faster with tex fonts turned off

# Read in data files and do basic processing

In [ ]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

# Create storage dictionaries and calculate elastic operators

In [ ]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly

# Elastic operators
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)
celeri.get_all_mesh_smoothing_matrices(meshes)

# Calculate additional non-elastic operators

In [ ]:
# Build all linear kinematic operators
operators.block_rotation = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)

# TODO: Update celeri.block_constraints to properly use block.
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

# Get additional matrix shape information for assembly
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct

In [ ]:
def plot_slip_distributions(mesh, slip_distribution_input, slip_distribution_estimated, suptitle_string):
    marker_size = 2.0
    plt.figure(figsize=(12, 7))

    plt.subplot(2, 2, 1)
    plt.title("input strike-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_input[0::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 2)
    plt.title("input dip-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_input[1::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 3)
    plt.title("estimated strike-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_estimated[0::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 4)
    plt.title("estimated dip-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_estimated[1::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.suptitle(suptitle_string)
    plt.show()


def get_synthetic_displacements(mesh, tri_linear_operator):
    """
    Prescribe dip-slip in a Gaussian pattern.
    """
    tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - np.mean(mesh.centroids[:, 0])
    tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - np.mean(mesh.centroids[:, 1])

    # Hardcoded northern Cascadia example that Jack suggested.
    tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - 234.5
    tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - 48.5

    tri_centroid_to_mesh_centroid_distance = np.sqrt(tri_centroid_to_mesh_lon ** 2 + tri_centroid_to_mesh_lat ** 2)
    dip_slip_distribution = np.exp(-(tri_centroid_to_mesh_centroid_distance / 1.0) ** 2.0)
    slip_distribution = np.zeros(2 * dip_slip_distribution.size)
    slip_distribution[1::2] = dip_slip_distribution # Dip slip only
    synthetic_displacements = tri_linear_operator @ slip_distribution
    return slip_distribution, synthetic_displacements


# Shrink operators.tri_station so that there are no vertical displacments and no tensile slip
tde_matrix = copy.deepcopy(operators.tri_station)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[0], 3), axis=0)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[1], 3), axis=1)

# Generate Guassian slip source and synthetic displacements
slip_distribution, synthetic_displacements = get_synthetic_displacements(meshes[0], tde_matrix)

# Slip estimation with direct inverse and smoothing matrix
smoothing_matrix = meshes[0].smoothing_matrix.toarray()
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[0], 3), axis=0)
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[1], 3), axis=1)
smoothing_matrix = meshes[0].smoothing_weight * smoothing_matrix # Weight smoothing matrix
tde_and_smoothing_matrix = np.vstack((tde_matrix, smoothing_matrix))
# synthetic_displacements_with_smoothing = np.hstack((synthetic_displacements, np.zeros(smoothing_matrix.shape[0]).T))
# estimated_slip_distribution = np.linalg.inv(tde_and_smoothing_matrix.T @ tde_and_smoothing_matrix) @ tde_and_smoothing_matrix.T @ synthetic_displacements_with_smoothing
# plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution, suptitle_string="Direct inverse - with smoothing")

# Sketching out the assembly of the block model system
The list here is in as of right now (09/19/21)...a work in progress

## The observation vector:    
| What | Symbolically | Variable name |
| - | - | - |
| GPS velocities | $\mathbf{v}$ | ```assembly.data.east_vel, assembly.data.north_vel``` |
| Slip rate constraints | $\dot{\mathbf{s}}$ | ```assembly.slip_rate_constraints``` |
| Rotation vector constraints | $\boldsymbol{\omega}$ | ```assembly.block_constraints``` |

## The state vector:    
| What | Symbolically | Variable name |
| - | - | - |
| Block rotation rates | $\boldsymbol{\Omega}$ | ```estimation.block_rotation_vectors``` |
| TDE slip | $\mathbf{t}$ | ```estimation.tri_slip_rates``` |
  
## Matrix operators:    
| What | Symbolically | Variable name |
| - | - | - |
| Rotation vector to GPS | $\boldsymbol{\Omega} \rightarrow \mathbf{v}$ | ```operators.block_rotation``` | 
| Rotation vector to GPS | $\boldsymbol{\Omega} \rightarrow \mathbf{v}$ | ```operators.global_float_block_rotation``` | 
| Rotation vector to segment slip rates| $\boldsymbol{\Omega} \rightarrow \dot{\mathbf{s}}$ | ```operators.slip_rate_segment_block``` | 
| Segment slip rates to elastic deformation | $\dot{\mathbf{s}} \rightarrow \mathbf{v}$ | ```operators.okada_segment_station``` | 
| TDE slip to GPS | $\mathbf{t} \rightarrow \mathbf{v}$ | ```operators.tri_station``` |
| Rotation vector to slip rate constraints| $\boldsymbol{\Omega} \rightarrow \dot{\mathbf{s}}$ | ```operators.slip_rate_constraints``` |
| Rotation vector to Euler pole contraints | $\mathbf{I}$ | ```operators.block_motion_contraints``` |



# Inversion with block motion only and no elastic deformation
-  Runs and solution looks reasonable

In [148]:
# Rotations only - with JDF a priori and no global float
data_vector = np.zeros(2 * assembly.data.n_stations
                     + 3 * assembly.data.n_block_constraints)
data_vector[0:2 * assembly.data.n_stations] = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel)

# Add block motion costraints to data vector
data_vector[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints] = assembly.data.block_constraints
data_vector[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints] = 0

operator = np.zeros((2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, 3 * len(block)))
operator[0:2 * assembly.data.n_stations, :] = np.delete(operators.block_rotation, np.arange(2, operators.block_rotation.shape[0], 3), axis=0) # Delete up velocity partials
operator[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, :]  = operators.block_motion_constraints

estimation = addict.Dict()
state_vector = np.linalg.inv(operator.T @ operator) @ operator.T @ data_vector 
estimation.predictions = operator @ state_vector
vel = estimation.predictions[0:2 * assembly.data.n_stations]
estimation.east_vel = vel[0::2]
estimation.north_vel = vel[1::2]
east_vel_rotation_only = vel[0::2]
north_vel_rotation_only = vel[1::2]


# Plot observed and estimated velocities
lon_min=235
lon_max=255
lat_min=30
lat_max=50
quiver_scale=1e2
plt.figure()
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
plt.quiver(station.lon, station.lat, estimation.east_vel, estimation.north_vel, scale=quiver_scale, scale_units="inches", color="r")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("observed and estimated velocities - block rotations only")
plt.show()

# Calculate mean squared residual velocity
east_vel_residual = estimation.east_vel - station.east_vel
north_vel_residual = estimation.north_vel - station.north_vel
residual = np.concatenate((east_vel_residual, north_vel_residual))
print(np.sum(residual**2) / len(station))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

12.59569094598103


# Inversion with block rotations and fully locked rectangular segments
- RUNS BUT LOOKS VERY INCORRECT AND INCREASES MISFIT

In [150]:
# Elastic slip deficit contribution to velocities with fully locked fault segments
slip_rate_to_okada_to_velocities = copy.deepcopy(operators.okada_segment_station)
rotation_to_slip_rate = copy.deepcopy(operators.slip_rate_segment_block)

# Delete rows associated with vertical velocities
slip_rate_to_okada_to_velocities = np.delete(slip_rate_to_okada_to_velocities, np.arange(2, slip_rate_to_okada_to_velocities.shape[0], 3), axis=0)

# Find indices of segments with:
# dip-slip on vertical faults
# tensile-slip on non-vertical faults
# vertical_segment_idx = np.where(segment.dip == 90)[0]
# non_vertical_segment_idx = np.where(segment.dip != 90)[0]

# # Create array to delete either dip- or tensile-slip entries
# # so that every fault has only 2 components of slip
# if vertical_segment_idx.size > 0 and non_vertical_segment_idx.size > 0:
#     delete_dip_slip_idx = 1 + 3 * vertical_segment_idx
#     delete_tensile_slip_idx = 2 + 3 * non_vertical_segment_idx
#     delete_idx = np.concatenate((delete_dip_slip_idx, delete_tensile_slip_idx))
# elif vertical_segment_idx.size == 0 and non_vertical_segment_idx.size > 0:
#     delete_idx = 1 + 3 * vertical_segment_idx
# elif vertical_segment_idx.size > 0 and non_vertical_segment_idx.size == 0:
#     delete_idx = 2 + 3 * non_vertical_segment_idx

# slip_rate_to_okada_to_velocities = np.delete(slip_rate_to_okada_to_velocities, delete_idx, axis=1)
# rotation_to_slip_rate = np.delete(rotation_to_slip_rate, delete_idx, axis=0)
rotation_to_slip_rate_to_okada_to_velocities = slip_rate_to_okada_to_velocities @ rotation_to_slip_rate

operator = np.zeros((2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, 3 * len(block)))
operator[0:2 * assembly.data.n_stations, :] = np.delete(operators.block_rotation, np.arange(2, operators.block_rotation.shape[0], 3), axis=0) - rotation_to_slip_rate_to_okada_to_velocities # Delete up velocity partials
operator[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, :] = operators.block_motion_constraints

estimation = addict.Dict()
state_vector = np.linalg.inv(operator.T @ operator) @ operator.T @ data_vector 
estimation.predictions = operator @ state_vector
vel = estimation.predictions[0:2 * assembly.data.n_stations]
estimation.east_vel = vel[0::2]
estimation.north_vel = vel[1::2]
east_vel_rotation_segment = vel[0::2]
north_vel_rotation_segment = vel[1::2]


# Plot observed and estimated velocities
lon_min=235
lon_max=255
lat_min=30
lat_max=50
quiver_scale=1e2
plt.figure()
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
plt.quiver(station.lon, station.lat, estimation.east_vel, estimation.north_vel, scale=quiver_scale, scale_units="inches", color="r")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("observed and estimated velocities - block rotations and elastic segments")
plt.show()

# Calculate mean squared residual velocity
east_vel_residual = estimation.east_vel - station.east_vel
north_vel_residual = estimation.north_vel - station.north_vel
residual = np.concatenate((east_vel_residual, north_vel_residual))
print(np.sum(residual**2) / len(station))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

29.548709653812278


# Plot elastic velocities only
- CLEARLY INCORRECT

In [151]:
# Calculate elastic velocities
elastic_velocities = rotation_to_slip_rate_to_okada_to_velocities @ state_vector

# Plot elastic velocities
lon_min=235
lon_max=255
lat_min=30
lat_max=50
quiver_scale=1e2
plt.figure()
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
# plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
plt.quiver(station.lon, station.lat, elastic_velocities[0::2], elastic_velocities[1::2], scale=quiver_scale, scale_units="inches", color="r")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("elastic velocity only -- CLEARLY WRONG")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# TODO: Plot estimated slip rates

# TODO: Plot elastic deformation for a single fault from partials operator?

In [ ]:
# Attempt to add global float

operator2 = np.zeros((2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, 3 * len(block) + 3)) # the "+3" at the end is for the global reference frame float
operator2[0:2 * assembly.data.n_stations, 0:-3] = np.delete(operators.block_rotation, np.arange(2, operators.block_rotation.shape[0], 3), axis=0) # Delete up velocity partials
operator2[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, 0:-3]  = operators.block_motion_constraints
operator2[0:2 * assembly.data.n_stations, -3:] = np.delete(operators.global_float_block_rotation, np.arange(2, operators.global_float_block_rotation.shape[0], 3), axis=0) # Global float


b = np.delete(operators.global_float_block_rotation, np.arange(2, operators.global_float_block_rotation.shape[0], 3), axis=0)
print(np.linalg.cond(b))

mat_to_plot = 1 * np.abs(operator2)>0.0
plt.figure()
plt.imshow(operator2, aspect="auto", interpolation="none")
plt.colorbar()
plt.show()

print(np.linalg.matrix_rank(operator))
print(np.linalg.matrix_rank(operator2))

assert(False)

estimation2 = addict.Dict()
state_vector2 = np.linalg.inv(operator2.T @ operator2) @ operator2.T @ data_vector 
estimation2.predictions = operator2 @ state_vector2
vel2 = estimation2.predictions[0:2 * assembly.data.n_stations]
estimation2.east_vel = vel2[0::2]
estimation2.north_vel = vel2[1::2]

# Isolate the part of the state vector assocated with the global float
state_vector3 = copy.deepcopy(state_vector2)
state_vector3[0:-3] = 0
estimation3 = addict.Dict()
estimation3.predictions = operator2 @ state_vector3
vel3 = estimation3.predictions[0:2 * assembly.data.n_stations]
estimation3.east_vel = vel3[0::2]
estimation3.north_vel = vel3[1::2]

# Plot observed and estimated velocities
lon_min=235
lon_max=255
lat_min=30
lat_max=50
quiver_scale=1e2

plt.figure()
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
plt.quiver(station.lon, station.lat, estimation2.east_vel, estimation2.north_vel, scale=quiver_scale, scale_units="inches", color="r")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("observed and estimated blocks + global float")
plt.show()

plt.figure()
quiver_scale=1e1

for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
# plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
plt.quiver(station.lon, station.lat, estimation3.east_vel, estimation3.north_vel, scale=quiver_scale, scale_units="inches", color="r")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("estimate global float")
plt.show()



In [ ]:
# Compare residuals
east_vel_residual = estimation.east_vel - station.east_vel
north_vel_residual = estimation.north_vel - station.north_vel
residual = np.concatenate((east_vel_residual, north_vel_residual))

east_vel_residual2 = estimation2.east_vel - station.east_vel
north_vel_residual2 = estimation2.north_vel - station.north_vel
residual2 = np.concatenate((east_vel_residual2, north_vel_residual2))

east_vel_residual3 = estimation3.east_vel - station.east_vel
north_vel_residual3 = estimation3.north_vel - station.north_vel
residual3 = np.concatenate((east_vel_residual3, north_vel_residual3))

print(np.sum(residual**2))
print(np.sum(residual2**2))
print(np.sum(residual3**2))

num_bins = 100
plt.figure()
n, bins, _ = plt.hist(residual, num_bins, density=True, histtype='step', label="no global float")
n, bins, _ = plt.hist(residual2, num_bins, density=True, histtype='step', label="global float")
n, bins, _ = plt.hist(residual3, num_bins, density=True, histtype='step', label="only global float")
plt.legend()
plt.show()